In [4]:
#借鉴github上wclemens1实现的ConvAC熟悉一下该网络的构建 
#https://github.com/wclemens1/CNN_vs_ConvAC/blob/master/CNNvsConvAC.ipynb
#对比CNN与ConvAC
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Model,load_model
from keras.layers import Dense, Dropout, Lambda, Flatten, Conv2D,MaxPool2D, Input, AveragePooling2D
from keras.optimizers import Adam, RMSprop

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
from keras.engine.topology import Layer
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

assert K.image_data_format() == "channels_last"

#to_categorical() 用于将类别标签转换为one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train = np.reshape(X_train,(-1,28,28,1))/255
X_test = np.reshape(X_test,(-1,28,28,1))/255

In [7]:
X_train.shape

(60000, 28, 28, 1)

In [14]:
#定义迭代算法和数据增强的内容
optimizer = RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.0)

#进行数据增强，提高模型的泛化能力
datagen = ImageDataGenerator(featurewise_center=False,#输入均值置零零
                            samplewise_center=False,#样本均值置零
                            featurewise_std_normalization=False,#通过数据集标准差划分输入
                            samplewise_std_normalization=False,#通过输入的标准差划分输入
                            zca_whitening=False,#使用ZCA白化
                            rotation_range=10,#随机旋转图片的角度（0 to 180）
                            zoom_range=0.1,#
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            horizontal_flip=False,
                            vertical_flip=False)
datagen.fit(X_train)
'''
data_iter = datagen.flow(X_train,y_train,batch_size=8)
while True:
    x_batch,y_batch = data_iter.next()
    for i in range(8):
        print(i//4)
        plt.subplot(2,4,i+1)
        plt.imshow(x_batch[i].reshape(28,28),cmap='gray')
    plt.show()
'''

"\ndata_iter = datagen.flow(X_train,y_train,batch_size=8)\nwhile True:\n    x_batch,y_batch = data_iter.next()\n    for i in range(8):\n        print(i//4)\n        plt.subplot(2,4,i+1)\n        plt.imshow(x_batch[i].reshape(28,28),cmap='gray')\n    plt.show()\n"

In [17]:
'''
搭建一个简单的cnn网络
'''
input_image = Input(shape = (28,28,1))

X1 = Conv2D(32,(3,3),activation="relu",padding="same")(input_image)
X1 = MaxPool2D(pool_size=(2,2))(X1)
X1 = Dropout(0.25)(X1)

X1 = Conv2D(64,(3,3),activation="relu",padding="same")(X1)
X1 = MaxPool2D(pool_size=(2,2))(X1)
X1 = Dropout(0.25)(X1)

X1 = Flatten()(X1)
X1 = Dense(256,activation="relu")(X1)
X1 = Dropout(0.5)(X1)
CNN_output = Dense(10,activation="softmax")(X1)

CNN_model = Model(input_image,CNN_output)
CNN_model.compile(optimizer=optimizer,
                 loss="categorical_crossentropy",
                 metrics = ["accuracy"])

In [19]:
epochs = 3
batch_size = 100
CNN_model.fit_generator(datagen.flow(X_train,y_train,batch_size=batch_size),
                       epochs = epochs,
                       verbose = 1,
                       steps_per_epoch = X_train.shape[0] // batch_size)

Epoch 1/3
600/600 [==============================] - 141s 234ms/step - loss: 0.1130 - acc: 0.9666
Epoch 2/3
600/600 [==============================] - 136s 227ms/step - loss: 0.0992 - acc: 0.9707
Epoch 3/3
600/600 [==============================] - 146s 243ms/step - loss: 0.0916 - acc: 0.9734


In [20]:
CNN_model.evaluate(x=X_test,y=y_test)

10000/10000 [==============================] - 6s 636us/step


[0.02589370626998134, 0.992]

In [21]:
X2 = Conv2D(32,(3,3),padding="same")(input_image)
X2 = MaxPool2D(pool_size=(2,2))(X2)
X2 = Lambda(lambda x:2*2*x)(X2)
X2 = Dropout(0.25)(X2)

X2 = Conv2D(64,(3,3),padding="same")(X2)
X2 = MaxPool2D(pool_size=(2,2))(X2)
X2 = Lambda(lambda x:2*2*x)(X2)
X2 = Dropout(0.25)(X2)

X2 = Flatten()(X2)
X2 = Dense(256,activation="relu")(X2)
X2 = Dropout(0.5)(X2)
ConvAC = Dense(10,activation="softmax")(X2)

ConvAC_model = Model(input_image,ConvAC)
ConvAC_model.compile(optimizer=optimizer,
                    loss = "categorical_crossentropy",
                    metrics = ["accuracy"])

In [22]:
ConvAC_model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                        epochs = epochs,
                        verbose = 1,
                        steps_per_epoch = X_train.shape[0] // batch_size)

Epoch 1/3
600/600 [==============================] - 138s 229ms/step - loss: 0.5051 - acc: 0.8468
Epoch 2/3
600/600 [==============================] - 135s 226ms/step - loss: 0.2271 - acc: 0.9334
Epoch 3/3
600/600 [==============================] - 149s 248ms/step - loss: 0.1916 - acc: 0.9451


In [23]:
ConvAC_model.evaluate(x=X_test,y=y_test)

10000/10000 [==============================] - 7s 666us/step


[0.05761433718447879, 0.9805]